# CBIR CON HARRIS

In [ ]:
import cv2
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Función para calcular los descriptores de esquinas Harris de una imagen
def calculate_harris_descriptors(image, max_corners=100):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detectar esquinas Harris
    dst = cv2.cornerHarris(gray_image, blockSize=2, ksize=3, k=0.04)

    # Normalizar la matriz de esquinas
    norm_dst = cv2.normalize(dst, None, 0, 255, cv2.NORM_MINMAX)

    # Encontrar las coordenadas de las esquinas
    corners = np.argwhere(norm_dst > 150)  # Puedes ajustar este umbral según tus necesidades

    # Tomar un máximo de 'max_corners' esquinas
    if len(corners) > max_corners:
        np.random.shuffle(corners)
        corners = corners[:max_corners]

    return corners

# Cargar la imagen de consulta
query_image = cv2.imread('n04487081_3.JPEG')
query_corners = calculate_harris_descriptors(query_image)

# Crear una lista de descriptores de esquinas Harris para la base de datos
database_images = []
for i in range(100):
    image_path = f'n04487081_{i}.JPEG'
    image = cv2.imread(image_path)
    database_corners = calculate_harris_descriptors(image)
    database_images.append((image, database_corners))

# Crear un modelo de k-NN
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')  # Puedes ajustar el número de vecinos (n_neighbors) y la métrica según tus necesidades

# Ajustar el modelo k-NN a los descriptores de esquinas Harris de la base de datos
# Ajustar el modelo k-NN a los descriptores de esquinas Harris de la base de datos

knn.fit(database_corners)

# Encontrar las imágenes más similares usando k-NN

query_features = np.mean(query_corners, axis=0)  # Media de las coordenadas de las esquinas
query_features = query_features.reshape(1, -1)  # Reformatear a una matriz bidimensional

# Encontrar las imágenes más similares usando k-NN
distances, indices = knn.kneighbors(query_features)

# Mostrar las imágenes recuperadas más similares
display_n_similar(indices[0], 8)
